In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# Import the data by reading in the CSV file and setting the DatetimeIndex 
# Review the first 5 rows of the DataFrame
fund_df = pd.read_csv(
    Path('Resources/whale_navs.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True
)
fund_df.columns = [name.lower().replace(' ', '_') for name in fund_df.columns]
fund_df.head()

,soros_fund_management_llc,paulson_&_co.inc.,tiger_global_management_llc,berkshire_hathaway_inc,s&p_500
date,,,,,
2014-10-01,31.950240,14.991826,59.977830,51.948712,194.35
2014-10-02,31.936110,14.994072,59.978626,51.957619,194.38
2014-10-03,31.969707,14.999596,60.002264,52.022484,196.52
2014-10-06,32.048215,14.999471,60.006244,52.036387,196.29
2014-10-07,31.964216,14.994720,59.993735,52.005864,193.26


In [3]:
# Prepare for the analysis by converting the dataframe of NAVs and prices to daily returns
# Drop any rows with all missing values
# Review the first five rows of the daily returns DataFrame.
daily_returns = fund_df.pct_change().dropna(axis=0, how='all').copy()
daily_returns.head()

,soros_fund_management_llc,paulson_&_co.inc.,tiger_global_management_llc,berkshire_hathaway_inc,s&p_500
date,,,,,
2014-10-02,-0.000442,0.000150,0.000013,0.000171,0.000154
2014-10-03,0.001052,0.000368,0.000394,0.001248,0.011009
2014-10-06,0.002456,-0.000008,0.000066,0.000267,-0.001170
2014-10-07,-0.002621,-0.000317,-0.000208,-0.000587,-0.015436
2014-10-08,0.000390,0.000369,0.000242,0.000842,0.017489


In [7]:
# Calculate the annual average return data for the for fund portfolios and the S&P 500
# Use 252 as the number of trading days in the year
# Review the annual average returns sorted from lowest to highest
avg_annl_rtn = (daily_returns.mean() * 252)

# Calculate the annualized Sharpe Ratios for each of the 4 portfolios and the S&P 500.
# Review the Sharpe ratios sorted lowest to highest

# Calculate the annualized standard deviaton.
std_dev_annl = (daily_returns.std() * np.sqrt(252))

# Calculate the Sharpe Ratio.
risk_ratios_df = pd.DataFrame(
    avg_annl_rtn / std_dev_annl,
    columns=['sharpe_ratio']
)
risk_ratios_df

,sharpe_ratio
soros_fund_management_llc,-0.102290
paulson_&_co.inc.,-0.189998
tiger_global_management_llc,0.579002
berkshire_hathaway_inc,0.717512
s&p_500,0.589612


In [9]:
# Calculate the annualized standard deviation
# for losses only.
neg_std_dev_annl = (daily_returns[daily_returns < 0].std() * np.sqrt(252))

# Calculate the Sortino ratio
risk_ratios_df['sortino_ratio'] = avg_annl_rtn / neg_std_dev_annl
risk_ratios_df

,sharpe_ratio,sortino_ratio
soros_fund_management_llc,-0.102290,-0.110985
paulson_&_co.inc.,-0.189998,-0.223319
tiger_global_management_llc,0.579002,0.624359
berkshire_hathaway_inc,0.717512,0.937573
s&p_500,0.589612,0.665381


In [12]:
ratios_index = risk_ratios_df.index.to_list()[:-1]
ratios_index

['soros_fund_management_llc',
 'paulson_&_co.inc.',
 'tiger_global_management_llc',
 'berkshire_hathaway_inc']

In [13]:
test_series = pd.Series([1, 2, 3, 4], ratios_index)
test_series

soros_fund_management_llc      1
paulson_&_co.inc.              2
tiger_global_management_llc    3
berkshire_hathaway_inc         4
dtype: int64

In [14]:
risk_ratios_df['test'] = test_series

In [15]:
risk_ratios_df

,sharpe_ratio,sortino_ratio,test
soros_fund_management_llc,-0.102290,-0.110985,1.0
paulson_&_co.inc.,-0.189998,-0.223319,2.0
tiger_global_management_llc,0.579002,0.624359,3.0
berkshire_hathaway_inc,0.717512,0.937573,4.0
s&p_500,0.589612,0.665381,NaN
